In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [2]:
#!pip install census

In [3]:
hate_crime_csv = pd.read_csv("Resources/hate_crime.csv", low_memory=False)
hate_crime_df = pd.DataFrame(hate_crime_csv)
hate_crime_df.head()

,INCIDENT_ID,DATA_YEAR,ORI,PUB_AGENCY_NAME,PUB_AGENCY_UNIT,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,...,OFFENDER_RACE,OFFENDER_ETHNICITY,VICTIM_COUNT,OFFENSE_NAME,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,MULTIPLE_OFFENSE,MULTIPLE_BIAS
0,3015,1991,AR0040200,Rogers,NaN,City,AR,Arkansas,West South Central,South,...,White,NaN,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
1,3016,1991,AR0290100,Hope,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Simple Assault,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S
2,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
3,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
4,3017,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Service/Gas Station,Anti-White,Individual,S,S


### Reduce DF to include only neccessary columns.

In [4]:
columns = ['INCIDENT_ID', 'DATA_YEAR', 'INCIDENT_DATE', 'PUB_AGENCY_NAME', 'AGENCY_TYPE_NAME',
           'STATE_ABBR', 'STATE_NAME', 'POPULATION_GROUP_DESC', 'TOTAL_OFFENDER_COUNT',
           'TOTAL_INDIVIDUAL_VICTIMS', 'LOCATION_NAME', 'BIAS_DESC', 'VICTIM_TYPES'
          ]

reduced_hate_crime_df = hate_crime_df.loc[:,  columns]

# Filter Data: 2009

In [5]:
# Filter DF for 2009 only.
reduced_hate_crime_2009_df = reduced_hate_crime_df.loc[(reduced_hate_crime_df['DATA_YEAR'] == 2009)]
reduced_hate_crime_2009_df.head()

,INCIDENT_ID,DATA_YEAR,INCIDENT_DATE,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,POPULATION_GROUP_DESC,TOTAL_OFFENDER_COUNT,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES
136964,141003,2009,17-Feb-09,Anchorage,City,AK,Alaska,"Cities from 250,000 thru 499,999",1,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-American Indian or Alaska Native,Individual
136965,141004,2009,30-Jun-09,Anchorage,City,AK,Alaska,"Cities from 250,000 thru 499,999",0,1.0,Residence/Home,"Anti-Lesbian, Gay, Bisexual, or Transgender (M...",Individual
136966,141005,2009,5-Aug-09,Anchorage,City,AK,Alaska,"Cities from 250,000 thru 499,999",2,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-American Indian or Alaska Native,Individual
136967,141006,2009,28-Aug-09,Anchorage,City,AK,Alaska,"Cities from 250,000 thru 499,999",3,4.0,Highway/Road/Alley/Street/Sidewalk,Anti-American Indian or Alaska Native,Individual
136968,141007,2009,28-Aug-09,Anchorage,City,AK,Alaska,"Cities from 250,000 thru 499,999",2,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-American Indian or Alaska Native,Individual


In [6]:
# Create new DF for 2009 from value counts. Total number of hate crimes committed per state.

states2009 = reduced_hate_crime_2009_df['STATE_NAME'].value_counts(sort=True)
state_hatecrime_2009_df = pd.DataFrame(states2009)
state_hatecrime_2009_df = state_hatecrime_2009_df.reset_index()
state_hatecrime_2009_df.columns = ['State', 'Hate Crimes Committed']
state_hatecrime_2009_df.head()

,State,Hate Crimes Committed
0,California,1018
1,New York,641
2,New Jersey,549
3,Massachusetts,317
4,Michigan,315


In [7]:
# Census imports for 2012.

from census import Census
from us import states

# Census API Key
# Data not available for 2010 0r 2011.
from config import api_key
c = Census(api_key, year=2012)
#c.acs5.tables()

In [8]:
# Run Census Search to retrieve data on all states

census_data = c.acs5.get(("NAME", "B19013_001E", "B19301_001E", "B23025_005E", "B23025_002E",
                          "B17001_002E", "B17001_003E", "B17001_017E",
                          "B17001A_002E", "B17001B_002E", "B01003_001E",
                          "B02001_002E", "B02001_003E", "B15003_017E", "B15003_022E"), {'for': 'state:*'})

census_df = pd.DataFrame(census_data)

census_df = census_df.rename(columns={"B19301_001E": "Per Capita Income",
                                      "B19013_001E": "Median Household Income",
                                      "B23025_005E": "Unemployment Count",
                                      "B23025_002E": "Labor Force Size",
                                      "B17001_002E": "Poverty Count",
                                      "B17001_003E": "Poverty: Male",
                                      "B17001_017E": "Poverty: Female",
                                      "B17001A_002E": "Poverty: White",
                                      "B17001B_002E": "Poverty: Black",
                                      "B01003_001E": "Total Population",
                                      "B02001_002E": "Population: White",
                                      "B02001_003E": "Population: Black",
                                      "B15003_017E": "Education: High School",
                                      "B15003_022E": "Education: Bachelors",
                                      "NAME": "State", "state": "State Number"})

# Calculate & add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = 100 * \
    census_df["Poverty Count"].astype(
        int) / census_df["Total Population"].astype(int)

# Calculate & add in Unemployment Rate (Unemployment Count / Labor Force)
census_df["Unemployment Rate"] = 100 * \
    census_df["Unemployment Count"].astype(
        int) / census_df["Labor Force Size"].astype(int)

# Calculate and add share of population with at least a high school diploma.
census_df['Share of Population with HS Diploma'] = 1 - \
    census_df['Education: High School'].astype(
        int) / census_df['Total Population'].astype(int)

census_df = census_df[["State", "Total Population", "Median Household Income", "Per Capita Income", "Unemployment Rate",
                         "Poverty Rate", "Share of Population with HS Diploma"]]

census_df

,State,Total Population,Median Household Income,Per Capita Income,Unemployment Rate,Poverty Rate,Share of Population with HS Diploma
0,Alabama,4777326.0,43160.0,23587.0,10.188838,17.631035,0.828741
1,Alaska,711139.0,69917.0,32537.0,8.045603,9.369617,0.861117
2,Arizona,6410979.0,50256.0,25571.0,9.711096,16.767891,0.867727
3,Arkansas,2916372.0,40531.0,22007.0,8.521134,18.183860,0.808218
4,California,37325068.0,61400.0,29551.0,10.942019,14.976798,0.880476
5,Colorado,5042853.0,58244.0,31039.0,7.943789,12.569829,0.877578
6,Connecticut,3572213.0,69519.0,37807.0,9.116630,9.647129,0.832210
7,Delaware,900131.0,60119.0,29733.0,8.306160,11.182261,0.810472
8,District of Columbia,605759.0,64267.0,45004.0,10.410046,17.433666,0.887934
9,Florida,18885152.0,47309.0,26451.0,11.254820,15.287942,0.822082


In [9]:
# Read in GINI index for 2009.
gini2009_csv = pd.read_csv('Resources/GINI2009.csv')

# Merge csv with census DF on State.
census_df = pd.merge(census_df, gini2009_csv, how='left', on='State')

# Merge Hate Crime DF with census DF.
clean_hate_crime_2009_df = pd.merge(census_df, state_hatecrime_2009_df, how="left", on="State")

In [10]:
# Replace NaN values for states w/ no data (reported hate crimes) with 0.
# Hawaii is now the only stat w/ no data (reported hate crimes).
values = {'Hate Crimes Committed': 0}
clean_hate_crime_2009_df = clean_hate_crime_2009_df.fillna(value=values)
#clean_hate_crime_2009_df

In [13]:
# Drop Puerto Rico from DF.
# Drop Hawaii from DF.
clean_hate_crime_2009_df = clean_hate_crime_2009_df.drop([51])
clean_hate_crime_2009_df = clean_hate_crime_2009_df.drop([11])

In [32]:
# Calculate 2009 Hate Crime Rate per 100,000 total population.
# https://oag.ca.gov/sites/all/files/agweb/pdfs/cjsc/prof10/formulas.pdf
# Note:	 Calculating rates for geographies of less than 100,000 will generate an inflated rate
# when compared to geographies with populations of 100,000 or more; therefore,
# rates are not calculated for geographies with populations of less than 100,000.

clean_hate_crime_2009_df['Hate Crime Rate per 100,000 Population'] = 100000 * \
    clean_hate_crime_2009_df['Hate Crimes Committed'].astype(
        int) / clean_hate_crime_2009_df['Total Population'].astype(int)

# Sort descending on Hate Crime Rate.
clean_hate_crime_2009_df = clean_hate_crime_2009_df.sort_values(by='Hate Crime Rate', ascending=False)
clean_hate_crime_2009_df = clean_hate_crime_2009_df.reset_index(drop=True)
clean_hate_crime_2009_df

#clean_hate_crime_2009_df.to_csv('hate_crime_socioeconomic_2009.csv')
clean_hate_crime_2009_df

,State,Total Population,Median Household Income,Per Capita Income,Unemployment Rate,Poverty Rate,Share of Population with HS Diploma,Gini Index,Hate Crimes Committed,Hate Crime Rate,"Hate Crime Rate per 100,000 Population"
0,New Jersey,8793888.0,71637.0,35928.0,9.488355,9.672161,0.819232,0.465,549.0,6.242972,6.242972
1,District of Columbia,605759.0,64267.0,45004.0,10.410046,17.433666,0.887934,0.532,37.0,6.108040,6.108040
2,Connecticut,3572213.0,69519.0,37807.0,9.116630,9.647129,0.832210,0.480,200.0,5.598770,5.598770
3,South Dakota,815871.0,49091.0,25570.0,4.909060,13.276486,0.820752,0.452,42.0,5.147873,5.147873
4,Massachusetts,6560595.0,66658.0,35485.0,8.453433,10.643638,0.843351,0.468,317.0,4.831879,4.831879
5,Kansas,2851183.0,51273.0,26845.0,6.638647,12.824712,0.848229,0.444,122.0,4.278926,4.278926
6,Nebraska,1827306.0,51381.0,26523.0,5.543888,12.052935,0.836372,0.440,78.0,4.268579,4.268579
7,Colorado,5042853.0,58244.0,31039.0,7.943789,12.569829,0.877578,0.453,208.0,4.124649,4.124649
8,Delaware,900131.0,60119.0,29733.0,8.306160,11.182261,0.810472,0.434,37.0,4.110513,4.110513
9,Vermont,625498.0,54168.0,28846.0,6.527425,11.096918,0.810247,0.428,24.0,3.836943,3.836943


# Filter Data: 2017

In [17]:
# Filter DF for 2017 only.
reduced_hate_crime_2017_df = reduced_hate_crime_df.loc[(reduced_hate_crime_df['DATA_YEAR'] == 2017)]
reduced_hate_crime_2017_df.head()

,INCIDENT_ID,DATA_YEAR,INCIDENT_DATE,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,POPULATION_GROUP_DESC,TOTAL_OFFENDER_COUNT,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES
186860,279156,2017,29-Oct-17,Anchorage,City,AK,Alaska,"Cities from 250,000 thru 499,999",1,1.0,Amusement Park,Anti-White,Individual
186861,194491,2017,22-Dec-17,Juneau,City,AK,Alaska,"Cities from 25,000 thru 49,999",1,1.0,Residence/Home,Anti-White,Individual
186862,194492,2017,24-Oct-17,Juneau,City,AK,Alaska,"Cities from 25,000 thru 49,999",0,NaN,School-College/University,"Anti-Multiple Races, Group",Government
186863,194686,2017,4-May-17,State Troopers,State Police,AK,Alaska,"Non-MSA counties 100,000 or over",0,NaN,Church/Synagogue/Temple/Mosque,Anti-Other Christian,Religious Organization
186864,280686,2017,31-Jan-17,Hoover,City,AL,Alabama,"Cities from 50,000 thru 99,000",0,1.0,Grocery/Supermarket,Anti-Hispanic or Latino,Individual


In [18]:
# Create new DF for 2017 from value counts. Total number of hate crimes committed per state.
states2017 = reduced_hate_crime_2017_df['STATE_NAME'].value_counts(sort=True)
state_hatecrime_2017_df = pd.DataFrame(states2017)
state_hatecrime_2017_df = state_hatecrime_2017_df.reset_index()
state_hatecrime_2017_df.columns = ['State', 'Hate Crimes Committed']
state_hatecrime_2017_df.head()

,State,Hate Crimes Committed
0,California,1094
1,New York,554
2,Washington,511
3,New Jersey,499
4,Michigan,457


In [19]:
# Census imports for 2016.

c = Census(api_key, year=2016)

In [20]:
# Run Census Search to retrieve data on all states

census_data = c.acs5.get(("NAME", "B19013_001E", "B19301_001E", "B23025_005E", "B23025_002E",
                          "B17001_002E", "B17001_003E", "B17001_017E",
                          "B17001A_002E", "B17001B_002E", "B01003_001E",
                          "B02001_002E", "B02001_003E", "B15003_017E", "B15003_022E"), {'for': 'state:*'})

census_df = pd.DataFrame(census_data)

census_df = census_df.rename(columns={"B19301_001E": "Per Capita Income",
                                      "B19013_001E": "Median Household Income",
                                      "B23025_005E": "Unemployment Count",
                                      "B23025_002E": "Labor Force Size",
                                      "B17001_002E": "Poverty Count",
                                      "B17001_003E": "Poverty: Male",
                                      "B17001_017E": "Poverty: Female",
                                      "B17001A_002E": "Poverty: White",
                                      "B17001B_002E": "Poverty: Black",
                                      "B01003_001E": "Total Population",
                                      "B02001_002E": "Population: White",
                                      "B02001_003E": "Population: Black",
                                      "B15003_017E": "Education: High School",
                                      "B15003_022E": "Education: Bachelors",
                                      "NAME": "State", "state": "State Number"})

# Calculate & add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = 100 * \
    census_df["Poverty Count"].astype(
        int) / census_df["Total Population"].astype(int)

# Calculate & add in Unemployment Rate (Unemployment Count / Labor Force)
census_df["Unemployment Rate"] = 100 * \
    census_df["Unemployment Count"].astype(
        int) / census_df["Labor Force Size"].astype(int)

# Calculate and add share of population with at least a high school diploma.
census_df['Share of Population with HS Diploma'] = 1 - \
    census_df['Education: High School'].astype(
        int) / census_df['Total Population'].astype(int)

census_df = census_df[["State", "Total Population", "Median Household Income", "Per Capita Income", "Unemployment Rate",
                         "Poverty Rate", "Share of Population with HS Diploma"]]

census_df.head()

,State,Total Population,Median Household Income,Per Capita Income,Unemployment Rate,Poverty Rate,Share of Population with HS Diploma
0,Alabama,4841164.0,44758.0,24736.0,8.240621,17.943329,0.828924
1,Alaska,736855.0,74444.0,34191.0,7.525813,9.883356,0.855916
2,Arizona,6728577.0,51340.0,26686.0,7.943898,17.323663,0.866442
3,Arkansas,2968472.0,42336.0,23401.0,6.831175,18.273071,0.810969
4,California,38654206.0,63783.0,31458.0,8.682885,15.533257,0.878971


In [21]:
# Read in GINI index for 2009.
gini2017_csv = pd.read_csv('Resources/GINI2017.csv')

# Merge csv with census DF on State.
census_df = pd.merge(census_df, gini2017_csv, how='left', on='State')

# Merge Hate Crime DF with census DF.
clean_hate_crime_2017_df = pd.merge(census_df, state_hatecrime_2017_df, how="left", on="State")
#clean_hate_crime_2017_df

In [22]:
# Replace NaN values for states w/ no data (reported hate crimes) with 0.
# Hawaii is now the only stat w/ no data (reported hate crimes).
values = {'Hate Crimes Committed': 0}
clean_hate_crime_2017_df = clean_hate_crime_2017_df.fillna(value=values)
#clean_hate_crime_2017_df

In [25]:
# Drop Puerto Rico from DF.
clean_hate_crime_2017_df = clean_hate_crime_2017_df.drop([51])
clean_hate_crime_2017_df = clean_hate_crime_2017_df.drop([11])

In [33]:
# Calculate 2017 Hate Crime Rate per 100,000 total population.
# https://oag.ca.gov/sites/all/files/agweb/pdfs/cjsc/prof10/formulas.pdf
# Note:	 Calculating rates for geographies of less than 100,000 will generate an inflated rate
# when compared to geographies with populations of 100,000 or more; therefore,
# rates are not calculated for geographies with populations of less than 100,000.

# Check Washington, D.C. on calculation, or if this is a result of massive increase in reported hate crimes!!!!!!

clean_hate_crime_2017_df['Hate Crime Rate per 100,000 Population'] = 100000 * \
    clean_hate_crime_2017_df['Hate Crimes Committed'].astype(
        int) / clean_hate_crime_2017_df['Total Population'].astype(int)

# Sort descending on Hate Crime Rate.
clean_hate_crime_2017_df = clean_hate_crime_2017_df.sort_values(by='Hate Crime Rate', ascending=False)
clean_hate_crime_2017_df = clean_hate_crime_2017_df.reset_index(drop=True)
clean_hate_crime_2017_df

#clean_hate_crime_2017_df.to_csv('hate_crime_socioeconomic_2017.csv')
clean_hate_crime_2017_df

,State,Total Population,Median Household Income,Per Capita Income,Unemployment Rate,Poverty Rate,Share of Population with HS Diploma,Gini Index,Hate Crimes Committed,Hate Crime Rate,"Hate Crime Rate per 100,000 Population"
0,District of Columbia,659009.0,72935.0,48781.0,8.654845,17.004320,0.891784,0.5281,194.0,29.438141,29.438141
1,Kentucky,4411989.0,44811.0,24802.0,7.504868,18.226224,0.821654,0.4784,378.0,8.567564,8.567564
2,Washington,7073146.0,62848.0,32999.0,6.753284,12.487456,0.871189,0.4561,511.0,7.224508,7.224508
3,Massachusetts,6742143.0,70954.0,38069.0,6.772892,10.988138,0.846865,0.4851,434.0,6.437122,6.437122
4,New Jersey,8915456.0,73702.0,37538.0,7.848944,10.648261,0.824021,0.4794,499.0,5.597022,5.597022
5,Vermont,626249.0,56104.0,30663.0,4.956633,11.125447,0.816666,0.4525,34.0,5.429150,5.429150
6,Michigan,9909600.0,50803.0,27549.0,8.485464,15.894345,0.827432,0.4671,457.0,4.611690,4.611690
7,Arizona,6728577.0,51340.0,26686.0,7.943898,17.323663,0.866442,0.4668,264.0,3.923564,3.923564
8,Oregon,3982267.0,53270.0,28822.0,8.056216,15.423953,0.866535,0.4593,148.0,3.716476,3.716476
9,Ohio,11586941.0,50674.0,27800.0,7.237560,14.955103,0.797004,0.4642,386.0,3.331337,3.331337


# Hate Crime: 2009

In [34]:
clean_hate_crime_2009_df.describe()

,Total Population,Median Household Income,Per Capita Income,Unemployment Rate,Poverty Rate,Share of Population with HS Diploma,Gini Index,Hate Crimes Committed,Hate Crime Rate,"Hate Crime Rate per 100,000 Population"
count,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,6.155520e+06,53039.680000,27867.320000,8.483381,14.021739,0.834003,0.452580,132.240000,2.456107,2.456107
std,6.874164e+06,8574.555181,4658.848349,1.907964,2.989209,0.027136,0.022322,181.865566,1.606616,1.606616
min,5.628030e+05,38882.000000,20670.000000,3.317506,8.182325,0.760106,0.402000,2.000000,0.067394,0.067394
25%,1.833100e+06,47088.500000,24669.000000,7.142008,11.854240,0.819301,0.435250,29.750000,1.125782,1.125782
50%,4.434886e+06,51327.000000,26534.000000,8.490517,14.158877,0.834731,0.453000,67.500000,2.293228,2.293228
75%,6.694184e+06,58224.000000,29937.000000,9.710563,16.671222,0.848299,0.466500,153.000000,3.415625,3.415625
max,3.732507e+07,72999.000000,45004.000000,12.574643,21.570551,0.887934,0.532000,1018.000000,6.242972,6.242972
